# Import

Install HuggingFace implementation of bert (https://huggingface.co/).

In [101]:
!pip install transformers

In [102]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
import sys
import time
import logging

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)

from transformers import *

In [103]:
# Set logger config for logging
logger = logging.getLogger('mylogger')
logger.setLevel(logging.DEBUG)
timestamp = time.strftime("%Y.%m.%d_%H.%M.%S", time.localtime())
fh = logging.FileHandler('log_model.txt')
fh.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] ## %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(ch)

In [104]:
# Set Random Seed
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Load Data

In [105]:
!ls ../input/nlp-getting-started

sample_submission.csv  test.csv  train.csv


In [106]:
train_df = pd.read_csv('../input/nlp-getting-started/train.csv')
test_df = pd.read_csv('../input/nlp-getting-started/test.csv')
submit_df = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [107]:
print('Train size:', train_df.shape)
print('Test size:', test_df.shape)

Train size: (7613, 5)
Test size: (3263, 4)


In [108]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [109]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [110]:
train_df.text[5]

'#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires'

In [111]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


# Process Input DataFrames

In [112]:
class InputFeature(object):
    """ A single training/test data class """
    def __init__(self, id, input_ids, masks, segments, label=None):
        self.id = id
        self.features = {
            'input_ids': input_ids,
            'input_mask': masks,
            'segment_ids': segments
        }
        self.label = label

Use Bert Pre-trained Model (Base, Uncased)

In [113]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Tokenize text, output padding masks and segment ids as well

In [114]:
def bert_encoder(text, max_len=512):
    """ Return embedded text vector as a list in max_len with a mask list"""
    text_token = tokenizer.tokenize(text)
    text_token = text_token[:max_len-2]
    text_token = ["[CLS]"] + text_token + ["[SEP]"]
    text_ids = tokenizer.convert_tokens_to_ids(text_token)
    text_ids += [0] * (max_len - len(text_token))
    pad_masks = [1] * len(text_token) + [0] * (max_len - len(text_token))
    segment_ids = [0] * len(text_token) + [0] * (max_len - len(text_token))
    
    return text_ids, pad_masks, segment_ids

Process train DataFrame

In [115]:
# If want to change
max_seq_length = 512

In [116]:
train_set = []

for index, row in train_df.iterrows():
    input_ids, masks, segments = bert_encoder(row.text, max_seq_length)
    train_set.append(InputFeature(row.id, input_ids, masks, segments, row.target))

# numpy array to split train and valid within Fold later
train_valid_input_ids = np.array([data.features['input_ids'] for data in train_set])
train_valid_input_masks = np.array([data.features['input_mask'] for data in train_set])
train_valid_segment_ids =np.array([data.features['segment_ids'] for data in train_set])
train_valid_labels = np.array([data.label for data in train_set])

Process test DataFrame

In [117]:
test_set = []

for index, row in test_df.iterrows():
    input_ids, masks, segments = bert_encoder(row.text, max_seq_length)
    train_set.append(InputFeature(row.id, input_ids, masks, segments))

# torch.tensor as test set does not need to split in Fold later
test_input_ids = torch.tensor([data.features['input_ids'] for data in test_set], dtype=torch.long)
test_input_masks = torch.tensor([data.features['input_mask'] for data in test_set], dtype=torch.long)
test_segment_ids = torch.tensor([data.features['segment_ids'] for data in test_set], dtype=torch.long)

# Define a model

In [118]:
# Very Simple Model, just adding a logistic regression on top of pretrained bert model
class SingleModel(nn.Module):
    
    def __init__(self, ):
        
        super(SimgpleModel, self).__init__()
        self.base_model = BertModel.from_pretrained('bert-base-uncased')
        self.fc1 = torch.nn.Linear(768, 1)
        
    def forward(self, ids, masks):
        
        x = self.base_model(ids, attention_mask=masks)[1]
        x = self.fc1(x)
        return x

In [119]:
# Complex Model by adding 5 horizontal dropout layers (multi-sample dropout)
class MultiSampleDropoutModel(nn.Module):
    def __init__(self, hidden_size=768, num_class=2):
        super(MultiSampleDropoutModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased',  
                                        output_hidden_states=True,
                                        output_attentions=True)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.weights = nn.Parameter(torch.rand(13, 1))
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])
        self.fc = nn.Linear(hidden_size, num_class)

    def forward(self, input_ids, input_mask, segment_ids):
        all_hidden_states, all_attentions = self.bert(input_ids, token_type_ids=segment_ids,
                                                                attention_mask=input_mask)[-2:]
        batch_size = input_ids.shape[0]
        ht_cls = torch.cat(all_hidden_states)[:, :1, :].view(13, batch_size, 1, 768)
        atten = torch.sum(ht_cls * self.weights.view(13, 1, 1, 1), dim=[1, 3])
        atten = F.softmax(atten.view(-1), dim=0)
        feature = torch.sum(ht_cls * atten.view(13, 1, 1, 1), dim=[0, 2])
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                h = self.fc(dropout(feature))
            else:
                h += self.fc(dropout(feature))
        h = h / len(self.dropouts)
        return h        

In [120]:
# Hyperparameters
learning_rate = 1e-5  
num_epochs = 3  
batch_size = 8  
patience = 2  
file_name = 'model'

In [121]:
# Define metrics
def metric(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    return acc, f1

# Train
Use `StratifiedKFold` to split data into `7 folds`

In [122]:
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
oof_train = np.zeros((len(train_df), 2), dtype=np.float32)
oof_test = np.zeros((len(test_df), 2), dtype=np.float32)

In [123]:
for fold, (train_indices, valid_indices) in enumerate(skf.split(train_valid_labels, train_valid_labels)):
    
    # Number of folds to iterrate
    if fold == 1:
        break

    logger.info('================     fold {}        ==============='.format(fold))
    
    # Train Data in Tensor
    train_input_ids = torch.tensor(train_valid_input_ids[train_indices], dtype=torch.long)
    train_input_mask = torch.tensor(train_valid_input_masks[train_indices], dtype=torch.long)
    train_segment_ids = torch.tensor(train_valid_segment_ids[train_indices], dtype=torch.long)
    train_label = torch.tensor(train_valid_labels[train_indices], dtype=torch.long)
    
    # Validation Data in Tensor
    valid_input_ids = torch.tensor(train_valid_input_ids[valid_indices], dtype=torch.long)
    valid_input_mask = torch.tensor(train_valid_input_masks[valid_indices], dtype=torch.long)
    valid_segment_ids = torch.tensor(train_valid_segment_ids[valid_indices], dtype=torch.long)
    valid_label = torch.tensor(train_valid_labels[valid_indices], dtype=torch.long)

    # Load data into TensorDataset
    train = torch.utils.data.TensorDataset(train_input_ids, train_input_mask, train_segment_ids, train_label)
    valid = torch.utils.data.TensorDataset(valid_input_ids, valid_input_mask, valid_segment_ids, valid_label)
    test = torch.utils.data.TensorDataset(test_input_ids, test_input_masks, test_segment_ids)

    # Use DataLoader to load data from Dataset in batches
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

    # Set Model
    model = MultiSampleDropoutModel()
    
    # Move model to GUP/CPU device
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    
    # Loss Function - use Cross Entropy as binary classification
    loss_fn = torch.nn.CrossEntropyLoss()

    # Optimizer - Adam with parameter groups
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=1e-6)
    
    # Set Train Mode
    model.train()

    # Initialize
    best_f1 = 0.
    valid_best = np.zeros((valid_label.size(0), 2))
    early_stop = 0
    
    for epoch in range(num_epochs):
        train_loss = 0.
        for i, batch in tqdm(enumerate(train_loader)):
            # Move batch data to device
            batch = tuple(t.to(device) for t in batch)
            # Bert input features and labels from batch
            x_ids, x_mask, x_sids, y_truth = batch
            
            # Feedforward prediction
            y_pred = model(x_ids, x_mask, x_sids)
            # Calculate Loss
            loss = loss_fn(y_pred, y_truth)
            # Reset gradient
            optimizer.zero_grad()
            # Backward Propagation
            loss.backward()
            # Update Weights
            optimizer.step()
            # Training Loss
            train_loss += loss.item() / len(train_loader)
            
            #logger.debug('train batch: %d, train_loss: %8f\n' % (i, train_loss))
    
        # Move to Evaluation Mode
        model.eval()
        
        # Initialize
        val_loss = 0.
        valid_preds_fold = np.zeros((valid_label.size(0), 2))
        
        with torch.no_grad():
            for i, batch in tqdm(enumerate(valid_loader)):
                batch = tuple(t.to(device) for t in batch)
                x_ids, x_mask, x_sids, y_truth = batch
                y_pred = model(x_ids, x_mask, x_sids).detach()
                val_loss += loss_fn(y_pred, y_truth).item() / len(valid_loader)
                valid_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()
                
                print('validation batch: {}, val_loss: {}, valid_preds_fold: {}'.format(i, val_loss, valid_preds_fold[i * batch_size:(i + 1) * batch_size]))
    
        # Calculate metrics
        acc, f1 = metric(train_valid_labels[valid_indices], np.argmax(valid_preds_fold, axis=1))
        
        # If improving, save the model. If not, count up for early stopping
        if best_f1 < f1:
            early_stop = 0
            best_f1 = f1
            valid_best = valid_preds_fold
            torch.save(model.state_dict(), 'model_fold_{}.bin'.format(fold))
        else:
            early_stop += 1
            

        logger.info(
            'epoch: %d, train loss: %.8f, valid loss: %.8f, acc: %.8f, f1: %.8f, best_f1: %.8f\n' %
            (epoch, train_loss, val_loss, acc, f1, best_f1))
        
        if device == 'cuda:0':
            torch.cuda.empty_cache()  
        
        # Early stop if it reaces patience number
        if early_stop >= patience:
            break

    # Once all epochs are done, take the best model of the fold
    test_preds_fold = np.zeros((len(test_df), 2))
    valid_preds_fold = np.zeros((valid_label.size(0), 2))
    
    # Load the best model
    model.load_state_dict(torch.load('model_fold_{}.bin'.format(fold)))
    # Set Evaluation Mode
    model.eval()
    
    # Prediction on the validation set
    with torch.no_grad():
        for i, batch in tqdm(enumerate(valid_loader)):
            batch = tuple(t.cuda() for t in batch)
            x_ids, x_mask, x_sids, y_truth = batch
            y_pred = model(x_ids, x_mask, x_sids).detach()
            valid_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()

    # Prediction on the test set
    with torch.no_grad():
        for i, batch in tqdm(enumerate(test_loader)):
            batch = tuple(t.cuda() for t in batch)
            x_ids, x_mask, x_sids = batch
            y_pred = model(x_ids, x_mask, x_sids).detach()
            test_preds_fold[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()

    # Check the metrics for the validation set
    valid_best = valid_preds_fold
    oof_train[valid_indices] = valid_best
    acc, f1 = metric(train_valid_labels[valid_indices], np.argmax(valid_best, axis=1))
    logger.info('epoch: best, acc: %.8f, f1: %.8f, best_f1: %.8f\n' % (acc, f1, best_f1))
    
    #oof_test += test_preds_fold / 7 # uncomment this for 7 folds
    oof_test += test_preds_fold / 1 # comment this line when training for 7 folds

[2020-02-01 13:45:43,466][INFO] ## ================     fold 0        ===============
[2020-02-01 13:45:43,466][INFO] ## ================     fold 0        ===============
[2020-02-01 13:45:43,466][INFO] ## ================     fold 0        ===============
[2020-02-01 13:45:43,466][INFO] ## ================     fold 0        ===============
[2020-02-01 13:45:43,466][INFO] ## ================     fold 0        ===============
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:60: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:89: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


validation batch: 0, val_loss: 0.0014007913805272458, valid_preds_fold: [[0.90873921 0.09126081]
 [0.93612921 0.06387078]
 [0.88758212 0.11241786]
 [0.81546944 0.18453053]
 [0.14845935 0.85154068]
 [0.03656323 0.96343672]
 [0.68243593 0.3175641 ]
 [0.6248275  0.3751725 ]]
validation batch: 1, val_loss: 0.002769647288496477, valid_preds_fold: [[0.00663781 0.99336219]
 [0.77262014 0.22737986]
 [0.48017287 0.51982713]
 [0.78866172 0.21133833]
 [0.06533835 0.93466169]
 [0.08819859 0.91180146]
 [0.92857528 0.07142466]
 [0.89587545 0.10412456]]
validation batch: 2, val_loss: 0.005222178723690284, valid_preds_fold: [[0.90573227 0.09426774]
 [0.81852728 0.1814727 ]
 [0.87089539 0.12910458]
 [0.12250561 0.87749439]
 [0.00505433 0.9949457 ]
 [0.00357952 0.9964205 ]
 [0.08021006 0.91978991]
 [0.05693388 0.94306612]]
validation batch: 3, val_loss: 0.007084912627282804, valid_preds_fold: [[0.00455528 0.99544477]
 [0.67812264 0.32187739]
 [0.82313037 0.17686968]
 [0.7620306  0.23796941]
 [0.0033602 

validation batch: 31, val_loss: 0.08639713180978802, valid_preds_fold: [[0.91992074 0.08007926]
 [0.9235481  0.07645182]
 [0.86583126 0.13416873]
 [0.70940816 0.29059187]
 [0.90403759 0.09596247]
 [0.77913243 0.22086754]
 [0.03724131 0.96275872]
 [0.90547675 0.09452324]]
validation batch: 32, val_loss: 0.08809240766032768, valid_preds_fold: [[0.85921174 0.14078821]
 [0.00989301 0.99010694]
 [0.07435723 0.92564279]
 [0.00827257 0.99172747]
 [0.04733323 0.95266676]
 [0.0167468  0.98325318]
 [0.11205936 0.88794065]
 [0.75986093 0.24013908]]
validation batch: 33, val_loss: 0.09170700386710409, valid_preds_fold: [[0.00820702 0.99179298]
 [0.9098599  0.09014013]
 [0.75531715 0.24468282]
 [0.90765142 0.09234857]
 [0.79354751 0.20645255]
 [0.02911306 0.97088695]
 [0.0121697  0.98783022]
 [0.84462368 0.15537639]]
validation batch: 34, val_loss: 0.0951075208556913, valid_preds_fold: [[0.67753744 0.32246259]
 [0.77637976 0.22362018]
 [0.02721869 0.97278124]
 [0.91380984 0.08619022]
 [0.93239558 0

validation batch: 62, val_loss: 0.18138631982524894, valid_preds_fold: [[0.82726794 0.172732  ]
 [0.85341543 0.1465845 ]
 [0.87073517 0.12926485]
 [0.7252171  0.27478293]
 [0.35356873 0.64643127]
 [0.70368314 0.29631692]
 [0.29681653 0.70318347]
 [0.07430372 0.92569631]]
validation batch: 63, val_loss: 0.18614607657829335, valid_preds_fold: [[0.87349051 0.12650955]
 [0.15436259 0.84563738]
 [0.66682607 0.33317393]
 [0.14588073 0.8541193 ]
 [0.01478257 0.98521739]
 [0.00578671 0.99421328]
 [0.36905715 0.63094288]
 [0.60886753 0.39113241]]
validation batch: 64, val_loss: 0.18854058746003752, valid_preds_fold: [[0.00967501 0.99032497]
 [0.17086287 0.82913709]
 [0.03250336 0.96749663]
 [0.1339     0.86610001]
 [0.0196152  0.98038483]
 [0.02919033 0.97080964]
 [0.79457229 0.20542766]
 [0.90104514 0.09895479]]
validation batch: 65, val_loss: 0.18928690401524517, valid_preds_fold: [[0.71630138 0.28369862]
 [0.87199217 0.1280079 ]
 [0.04127893 0.95872104]
 [0.00952358 0.99047637]
 [0.00582559 

...

[2020-02-01 14:05:38,911][INFO] ## epoch: 2, train loss: 0.17690339, valid loss: 0.44168579, acc: 0.83471074, f1: 0.82911869, best_f1: 0.83905255

[2020-02-01 14:05:38,911][INFO] ## epoch: 2, train loss: 0.17690339, valid loss: 0.44168579, acc: 0.83471074, f1: 0.82911869, best_f1: 0.83905255

[2020-02-01 14:05:38,911][INFO] ## epoch: 2, train loss: 0.17690339, valid loss: 0.44168579, acc: 0.83471074, f1: 0.82911869, best_f1: 0.83905255

[2020-02-01 14:05:38,911][INFO] ## epoch: 2, train loss: 0.17690339, valid loss: 0.44168579, acc: 0.83471074, f1: 0.82911869, best_f1: 0.83905255

[2020-02-01 14:05:38,911][INFO] ## epoch: 2, train loss: 0.17690339, valid loss: 0.44168579, acc: 0.83471074, f1: 0.82911869, best_f1: 0.83905255



validation batch: 135, val_loss: 0.441418501854378, valid_preds_fold: [[7.61883020e-01 2.38116950e-01]
 [9.92913425e-01 7.08654383e-03]
 [9.21058834e-01 7.89411962e-02]
 [9.80051398e-01 1.99485458e-02]
 [8.79889471e-04 9.99120057e-01]
 [9.44732048e-04 9.99055326e-01]
 [3.55578819e-03 9.96444166e-01]
 [1.05204713e-03 9.98947918e-01]]
validation batch: 136, val_loss: 0.44168578996493, valid_preds_fold: [[0.03595602 0.96404392]]



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:133: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:141: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


[2020-02-01 14:06:00,001][INFO] ## epoch: best, acc: 0.84481175, f1: 0.83905255, best_f1: 0.83905255

[2020-02-01 14:06:00,001][INFO] ## epoch: best, acc: 0.84481175, f1: 0.83905255, best_f1: 0.83905255

[2020-02-01 14:06:00,001][INFO] ## epoch: best, acc: 0.84481175, f1: 0.83905255, best_f1: 0.83905255

[2020-02-01 14:06:00,001][INFO] ## epoch: best, acc: 0.84481175, f1: 0.83905255, best_f1: 0.83905255

[2020-02-01 14:06:00,001][INFO] ## epoch: best, acc: 0.84481175, f1: 0.83905255, best_f1: 0.83905255



In [125]:
logger.info(f1_score(train_df.target, np.argmax(oof_train, axis=1)))
train_df['pred_target'] = np.argmax(oof_train, axis=1)

[2020-02-01 14:14:05,573][INFO] ## 0.1937059142702116
[2020-02-01 14:14:05,573][INFO] ## 0.1937059142702116
[2020-02-01 14:14:05,573][INFO] ## 0.1937059142702116
[2020-02-01 14:14:05,573][INFO] ## 0.1937059142702116
[2020-02-01 14:14:05,573][INFO] ## 0.1937059142702116


In [126]:
train_df.head()

,id,keyword,location,text,target,pred_target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0


In [127]:
test_df['target'] = np.argmax(oof_test, axis=1)
logger.info(test_df['target'].value_counts())

[2020-02-01 14:15:28,572][INFO] ## 0    3263
Name: target, dtype: int64
[2020-02-01 14:15:28,572][INFO] ## 0    3263
Name: target, dtype: int64
[2020-02-01 14:15:28,572][INFO] ## 0    3263
Name: target, dtype: int64
[2020-02-01 14:15:28,572][INFO] ## 0    3263
Name: target, dtype: int64
[2020-02-01 14:15:28,572][INFO] ## 0    3263
Name: target, dtype: int64


In [129]:
submit_df['target'] = np.argmax(oof_test, axis=1)
submit_df.to_csv('submission_1fold.csv', index=False)

In [130]:
submit_df.head()

NameError: name 'submit' is not defined